In [ ]:
import numpy as np
from numpy.fft import fft2, ifft2
import matplotlib.pyplot as plt
import cv2

import time

In [25]:
# Physical Properties
M = 1.0       # Mobility
kappa = 0.01  # Interface parameter

In [38]:
# Simulation Paremeters

N = 1024   # Grid size
L = 1.0   # Domain size
dx = L/N  # Grid spacing

dt = 1e-10      # Initial timestep
dt_min = 1e-10  # Smallest time step
dt_max = 1e-2   # Largest time step
steps = 1000     # Total simulation steps

# Prepare the real space dimensions
x = np.linspace(0, L, N, endpoint=False)
y = np.linspace(0, L, N, endpoint=False)
X, Y = np.meshgrid(x, y)

# Prepare the wavenumber space dimensions.
kx = 2 * np.pi * np.fft.fftfreq(N, dx)
ky = 2 * np.pi * np.fft.fftfreq(N, dx)
KX, KY = np.meshgrid(kx, ky)

# Multiplying by k2 and k4 represent the laplacian and di-laplacian in fourrier space.
k2 = KX**2 + KY**2
k4 = k2**2

In [ ]:
# Initial random state [-1,1].
np.random.seed(0)
c = np.random.rand(N, N) * 2 - 1

sim_states = [c.copy()]
current_step = 0

print("Starting simulation.")
sim_start_time = time.time()

# For saving states at logarithmically placed steps.
log_steps = np.logspace(0, np.log10(steps), 100).astype(int)
log_steps = np.unique(log_steps)

while current_step < steps:
    # Determine when to next log state.
    next_target = next((s for s in log_steps if s > current_step), steps)
    steps_to_do = next_target - current_step
    
    accepted = False
    while not accepted:

        # Whole stepping
        
        c_hat = fft2(c)
        mu = c**3 - c
        mu_hat = fft2(mu)
        c_hat_new = (c_hat - dt * M * k2 * mu_hat) / (1 + dt * M * kappa * k4)
        c_trial = np.real(ifft2(c_hat_new))
        
        # Half stepping

        half_dt = dt / 2.0
        c_hat = fft2(c)
        mu = c**3 - c
        mu_hat = fft2(mu)
        c_hat_half = (c_hat - half_dt * M * k2 * mu_hat) / (1 + half_dt * M * kappa * k4)
        c_half = np.real(ifft2(c_hat_half))

        c_half_hat = fft2(c_half)
        mu_half = c_half**3 - c_half
        mu_half_hat = fft2(mu_half)
        c_hat_half2 = (c_half_hat - half_dt * M * k2 * mu_half_hat) / (1 + half_dt * M * kappa * k4)
        c_ref = np.real(ifft2(c_hat_half2))

        # Error analysis

        err = np.linalg.norm(c_trial - c_ref) / (np.linalg.norm(c_ref) + 1e-10)

        # This code is pseudo-working. The idea here is that if the code is getting small enough 
        # errors we are fine to increase the dt a bit and if it's not fine enough then we should decrase it. 
        # Because of the way it runs I only have it increasing over time.
        if err > 1:
            dt = max(dt * 0.5, dt_min)
            print(f"Step {current_step}: error {err:.2e} too high, reducing dt to {dt:.2e}")
        elif err < 1e-4 and dt < dt_max:
            dt = min(dt * 2.0, dt_max)
            print(f"Step {current_step}: error {err:.2e} too low, increasing dt to {dt:.2e}")
        else:
            c = c_ref.copy()
            accepted = True
            current_step += 1
            sim_states.append(c.copy())

sim_end_time = time.time()
sim_duration = sim_end_time - sim_start_time
print(f"Simulation complete. Computed {steps} steps in {sim_duration:.2f} seconds.")
print(f"Stored {len(sim_states)} states for animation.")

Starting simulation.
Step 39: error 9.87e-05 too low, increasing dt to 2.00e-10
Step 59: error 9.54e-05 too low, increasing dt to 4.00e-10
Step 78: error 9.79e-05 too low, increasing dt to 8.00e-10
Step 97: error 9.94e-05 too low, increasing dt to 1.60e-09
Step 117: error 9.54e-05 too low, increasing dt to 3.20e-09
Step 136: error 9.63e-05 too low, increasing dt to 6.40e-09
Step 155: error 9.71e-05 too low, increasing dt to 1.28e-08
Step 174: error 9.53e-05 too low, increasing dt to 2.56e-08
Step 192: error 9.70e-05 too low, increasing dt to 5.12e-08
Step 211: error 9.80e-05 too low, increasing dt to 1.02e-07
Step 231: error 9.52e-05 too low, increasing dt to 2.05e-07
Step 250: error 9.53e-05 too low, increasing dt to 4.10e-07
Step 269: error 9.79e-05 too low, increasing dt to 8.19e-07
Step 289: error 9.94e-05 too low, increasing dt to 1.64e-06
Step 308: error 9.69e-05 too low, increasing dt to 3.28e-06
Step 329: error 9.75e-05 too low, increasing dt to 6.55e-06
Step 353: error 9.81e-0

In [ ]:
fps = 45
output_filename = 'cahn_hilliard_simulation_colorful.mp4'

first_state = sim_states[0]
norm_state = (first_state - first_state.min()) / (first_state.max() - first_state.min()) * 255
gray_frame = norm_state.astype(np.uint8)
colored_frame = cv2.applyColorMap(gray_frame, cv2.COLORMAP_JET)
height, width, _ = colored_frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_filename, fourcc, fps, (width, height))

for state in sim_states:
    norm_state = (state - state.min()) / (state.max() - state.min()) * 255
    gray_frame = norm_state.astype(np.uint8)
    colored_frame = cv2.applyColorMap(gray_frame, cv2.COLORMAP_JET)
    
    video_writer.write(colored_frame)

video_writer.release()
print(f"Video saved as '{output_filename}'")

Video saved as 'cahn_hilliard_simulation_colorful.mp4'
